In [46]:
import importlib
import sys
import pandas as pd
import scipy.io
import h5py
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from scipy.stats import zscore



sys.path.insert(0, "D:/PhD/tangent_space_analysis/")
                
import TSAfuncs as tsa
importlib.reload(tsa)

<module 'TSAfuncs' from 'D:\\PhD/tangent_space_analysis\\TSAfuncs.py'>

In [33]:
ROI = scipy.io.loadmat('filteredROI.mat')
ROI = ROI['ROItokeep']
nparcels = [100, 400, 1000]
ROI_to_keep = dict()
for i,n in enumerate(nparcels):
    ROI_to_keep[n] = ROI[0][i][0]

In [54]:
df = pd.read_csv('allRecordings.csv')

#df['filepath'] = df['filepath'].str.replace("/scratch2/DavorCuric", "D:\PhD\CANBIND")
nparcels= 100
df = df[df['numparcels'] == nparcels]
df = df.dropna(how = 'any')
df.reset_index(drop = True, inplace = True)
bold_data = np.zeros([np.shape(df)[0], len(ROI_to_keep[nparcels]), 298])

for index,row in df.iterrows():
    filename = row['filepath'] +'/' + row['filename']
    filename = filename.replace( '/scratch2/DavorCuric/CANBIND/' ,"D:/PhD/CANBIND/")
        
    with h5py.File(filename, 'r') as f:
        ts = np.transpose(np.array(f['parc_data']))#scipy.io.loadmat(filename)
    ts = ts[ROI_to_keep[nparcels]-1,:] #ROI to keep was importet from matlab so include the -1 to get correct index
    bold_data[index, :,:] = ts
    


In [55]:
confounds = pd.get_dummies(df['subject'], drop_first=True)
confounds_matrix = confounds.values

In [67]:
bold_corrected = np.zeros_like(bold_data)

for roi in range(bold_data.shape[1]):
    y = bold_data[:, roi, :]
    reg = LinearRegression()
    reg.fit(confounds_matrix, y)
    residuals = y - 1*reg.predict(confounds_matrix)
    bold_corrected[:,roi,:] = residuals
    


In [ ]:
FCs = list()
for index,row in df.iterrows():
    ts = zscore(bold_corrected[index,:,:], axis = 1)
    fc = np.corrcoef(ts)
    
    subject_id = str(row['subject'])
    state = int(row['controls'])*'controls' + int(row['nonresponders'])*'nonresponders' + int(row['responders'])*'responders'
    session = row['session']

    if state == 'controls':
        continue
    if state == '':
        continue

    FCs.append(tsa.FC(np.array(fc), subject = subject_id, state = state, session = session))


In [69]:
regvals = np.array([1000])#0.01, 0.1, 1, 10 ])
trt = tsa.test_retest(FCs)

for r in regvals:
    score = trt.ts_test_retest(regvals = np.array([r]))#, refinv = 'logm')
    #score_classic = trt.classic_test_retest()
    #print('tangent space score = ' + str(score) + ', non-tangent space score = ' + str(score_classic))